In [1]:
!pip -q install -U ollama

In [13]:
from openai import OpenAI
import base64
import os
from ollama import Client
from google.colab import userdata
os.environ['OLLAMA_API_KEY']=userdata.get('OLLAMA_KEY')
client = OpenAI(
    base_url="https://ollama.com/v1/",
    api_key=os.environ.get('OLLAMA_API_KEY'),
    timeout=200.0,

)

In [8]:
import pandas as pd
seed_data = pd.read_csv('/content/drive/MyDrive/data/filtered_eco_hr_kr_df.csv')
seed_data.head()

,id,output_0,output_1,output_2,answer,reasoning_0,category,pred_0,pred_1,pred_2,num_chosen,problems,paragraph
0,generation-for-nlp-0,"[Reasoning]\n- 제시문은 “현종실록”에 기록된 상소 내용이며, 상소한 인...","[Reasoning]\n- 제시문은 현종실록에서 인용된 상소 내용으로, 상소한 인물...","[Reasoning]\n- 제시문은 현종실록에 기록된 상소문으로, 상소한 인물이 ‘...",2,"Okay, let's tackle this question step by step....",한국사,2,4,4,1,{'question': '상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두...,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)..."
1,generation-for-nlp-2,"[Reasoning]\n- 제시문은 고려 시대의 문헌으로 보이며, “삼한 산천의 음...","[Reasoning]\n- 제시문은 고려시대 문헌 <고려사>에서 인용된 것으로, ‘...",[Reasoning]\n- 제시문은 “나는 삼한 산천의 음덕을 입어 대업을 이루었다...,4,NaN,한국사,3,1,1,0,"{'question': '(가) 지역에 대한 설명으로 옳은 것은?', 'choice...",나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...
2,generation-for-nlp-4,"[Reasoning]\n- 제시문에서 (가)는 ""선비들 수만 명이 대궐 앞에 모여 ...",[Reasoning]\n- 제시문은 조선 말기의 개혁 정책과 관련된 역사적 사건을 ...,"[Reasoning]\n- 제시문에서 ""(가)이/가 크게 노하여... 천여 곳의 서...",3,NaN,한국사,4,2,1,0,"{'question': '(가) 인물이 추진한 정책으로 옳지 않은 것은?', 'ch...","선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가..."
3,generation-for-nlp-5,[Reasoning]\n- 제시문에서 “(가)의 사신 고제덕 등이 일본에 와서 왕이...,[Reasoning]\n- 제시문에서 “나(대무 예)는 큰 나라를 맡아 여러 주변국...,"[Reasoning]\n- 제시문에서 ""(가)의 사신 고제덕 등이 일본에 와서 왕이...",3,NaN,한국사,4,4,3,1,"{'question': '(가) 국가에 대한 설명으로 옳은 것은?', 'choice...",(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...
4,generation-for-nlp-6,[Reasoning]\n- 단계 1: 각 사건의 역사적 시기를 파악한다.\n - ...,[Reasoning]\n- 이 문제는 고대 한국사에서 삼국 시대 주요 사건의 시기 ...,"[Reasoning]\n이 문제는 고대 한국 역사에서 신라, 백제, 고구려의 주요 ...",4,NaN,한국사,4,-1,4,2,"{'question': '다음 사건을 시기 순으로 바르게 나열한 것은?', 'cho...",(가)신라의 한강 유역 확보 (나)관산성 전투(다) 백제의 웅진 천도 (라)고구려의...


In [9]:
seed_data.shape

(156, 13)

In [10]:
import re
import json
def extract_answer_num(text):
    """
    텍스트 덩어리(reasoning 포함) 안에서 {"Answer": "3"} 형태의 숫자만 추출
    """
    try:
        # 정규표현식으로 JSON 부분 탐색
        match = re.search(r'\{"Answer":\s*"(\d+)"\}', text)
        if match:
            return int(match.group(1))

        # 백업: JSON 객체 전체 탐색
        match_json = re.search(r'\{.*?\}', text, re.DOTALL)
        if match_json:
            data = json.loads(match_json.group(0))
            return int(data.get("Answer"))

        return None
    except:
        return None

In [14]:
import ast
import time
import json
import re
import pandas as pd
# from openai import OpenAI # client 설정은 기존 환경에 맞게 유지

# =========================
# 1. SYSTEM MESSAGES (CoT 강화 버전)
# =========================

# [핵심 변경] 모델에게 '생각(Reasoning)'을 먼저 하도록 강제합니다.
ANSWER_SYSTEM_MSG = """
당신은 객관식 문제의 출제 원리를 꿰뚫어 보고, 엄밀한 논리에 기반해 정답을 도출하는 전문가입니다.

문제를 해결할 때 다음 내부 절차를 반드시 따르세요:

1) [핵심 전제 파악] 판단에 앞서, 문제 해결에 필요한 핵심 개념·원리·공식·역사적 사실을 먼저 정의한다.
2) [독립적 검증] 각 선지를 서로 독립적으로 분석하며, 다른 선지와의 비교가 아닌 해당 선지 자체의 참/거짓 여부를 팩트와 논리에 근거해 서술한다.
3) [판단 기준] 지엽적인 사례나 예외적인 상황이 아니라, 문제에서 묻고자 하는 **핵심 원리, 보편적 인과 관계, 시대적/상황적 맥락**을 기준으로 판단한다.
4) [소거 및 확정] 모든 선지를 검증한 뒤, 논리적 모순이나 팩트 오류가 있는 선지를 제거하여 최종 정답을 확정한다.

※ 판단 과정에서 다음 논리적 오류를 피하라:
- 부분적 참을 전체의 참으로 확대 해석하는 오류 (성급한 일반화)
- 선후 관계를 인과 관계로 혼동하거나, 원인과 결과를 뒤집는 오류 (경제/과학 등)
- 특정 인물/사건의 단면만으로 전체 시기나 집단의 성격을 규정하는 오류 (역사)

출력 규칙을 반드시 지키세요.

[Reasoning]
- 핵심 지식: 이 문제 해결을 위해 전제되어야 할 개념, 원리, 또는 역사적 배경 (2~3문장)
- 판단 기준: 제시문에서 정답 도출의 근거가 되는 핵심 단서 (Key Clue)
- 선지별 판정:
  1번: [핵심 지식/단서] → [논리적 연결] → 판정 (정답/오답)
  2번: [핵심 지식/단서] → [논리적 연결] → 판정 (정답/오답)
  3번: [핵심 지식/단서] → [논리적 연결] → 판정 (정답/오답)
  4번: [핵심 지식/단서] → [논리적 연결] → 판정 (정답/오답)
- 판단 점검: 오답 매력도가 높은 선지가 있다면, 그것이 왜 답이 될 수 없는지 결정적 이유 서술

[Answer]
반드시 맨 마지막 줄에만 아래 JSON을 출력:
{"Answer":"번호"}
"""

# 둘 다 정답을 맞췄을 때, 어떤 '추론'이 더 논리적인지 평가하는 심판
JUDGE_SYSTEM_MSG = """
당신은 AI 모델의 답변 품질을 평가하는 채점관입니다.
두 모델 모두 정답을 맞췄지만, 추론 과정(Reasoning)의 논리성과 명확성이 더 뛰어난 답변을 고르세요.

[평가 기준]
1. 근거가 제시문과 사실에 부합하는가?
2. 오답 선지에 대한 분석이 포함되어 있는가?
3. 논리적 비약 없이 단계적으로 결론에 도달했는가?

더 나은 답변을 하나만 출력하세요:
A
B
TIE
"""

# =========================
# 2. HELPER FUNCTIONS
# =========================

def call_llm(messages, model_name, **kwargs):
    # 기본 파라미터
    default_params = {
        "max_tokens": 16382,
        "temperature": 0.7,
        "top_p": 0.95,
    }

    # extra_body 기본값
    default_extra_body = {}

    # kwargs에서 extra_body 분리
    extra_body = kwargs.pop("extra_body", {})
    extra_body = {**default_extra_body, **extra_body}

    # 일반 파라미터 병합
    params = {**default_params, **kwargs}

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=messages,
            **params,
            extra_body=extra_body if extra_body else None
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        print(f"LLM Call Error: {e}")
        return ""








# =========================
# 3. MAIN LOOP (Data Generation)
# =========================

dpo_results = []

# df_hist은 이미 로드되어 있다고 가정
for i in range(len(seed_data)):
    try:
        # 데이터 파싱
        problem = ast.literal_eval(seed_data.loc[i, "problems"])

        # Gold 정답 확보 (데이터셋 형식에 따라 int 변환 주의)
        gold = int(problem["answer"])

        # 프롬프트 구성
        user_content = f"""<제시문>
{seed_data.loc[i, "paragraph"]}
<질문>
{problem["question"]}
"""
        print(user_content)
        for k, choice in enumerate(problem["choices"]):
            # 선지 번호가 1부터 시작한다고 가정
            user_content += f"{k+1}. {choice}\n"

        base_messages = [
            {"role": "system", "content": ANSWER_SYSTEM_MSG},
            {"role": "user", "content": user_content}
        ]

        # ----- 모델 생성 (Temperature 차등) -----
        # A: 정석적인 답변 (낮은 온도)
        answer_a = call_llm(base_messages,'gemini-3-flash-preview:cloud', temperature=1.0)
        # B: 다양한 시도 (높은 온도)
        answer_b = call_llm(
    base_messages,
    "qwen3-vl:235b-instruct-cloud",
    temperature=0.6,
    top_p=0.8,
    extra_body={
        "topk": 20,
        "presence_penalty": 1.6
    }
)

        # 정답 번호만 추출 (검증용)
        ans_num_a = extract_answer_num(answer_a)
        ans_num_b = extract_answer_num(answer_b)

        # 파싱 실패 시 스킵
        if ans_num_a is None or ans_num_b is None:
            continue

        chosen = None
        rejected = None
        note = ""

        # =========================
        # SELECTION LOGIC
        # =========================

        # CASE 1: A만 정답
        if ans_num_a == gold and ans_num_b != gold:
            chosen = answer_a
            rejected = answer_b
            note = "A_Correct"

        # CASE 2: B만 정답
        elif ans_num_b == gold and ans_num_a != gold:
            chosen = answer_b
            rejected = answer_a
            note = "B_Correct"

        # CASE 3: 둘 다 정답 (Judge가 더 좋은 논리를 선택)
        elif ans_num_a == gold and ans_num_b == gold:
            judge_messages = [
                {"role": "system", "content": JUDGE_SYSTEM_MSG},
                {"role": "user", "content": f"### Question\n{user_content}\n\n### Answer A\n{answer_a}\n\n### Answer B\n{answer_b}"}
            ]
            verdict = call_llm(base_messages,'gemini-3-flash-preview:cloud', temperature=0.0)
            if "A" in verdict:
                chosen = answer_a
                rejected = answer_b
                note = "Both_Correct_Judge_A"
            elif "B" in verdict:
                chosen = answer_b
                rejected = answer_a
                note = "Both_Correct_Judge_B"
            else:
                # Judge가 판단 불가하면 안전하게 스킵하거나, A를 선택(TIE)
                print("Could not judge")
                continue

        # 둘 다 오답이면 스킵
        else:
          print("Both Wrong")
          continue

        # =========================
        # SAVE RESULT
        # =========================
        if chosen and rejected:
            # [중요] chosen/rejected에는 <reasoning>이 포함된 전체 텍스트가 들어감
            dpo_results.append({
                "prompt": user_content,
                "chosen": chosen,    # CoT + JSON
                "rejected": rejected, # CoT + JSON (오답 or 덜 좋은 추론)
                "gold": gold,
                "strategy": note
            })
            print(f"[{i}] Collected ({note}) | Total: {len(dpo_results)}")

    except Exception as e:
        print(f"[{i}] Error: {e}")

    # API 속도 조절
    time.sleep(1)

# =========================
# 저장
# =========================
df_dpo = pd.DataFrame(dpo_results)
df_dpo.to_json("cot_dpo_dataset3.jsonl", orient="records", lines=True, force_ascii=False)
print("완료! 저장된 데이터 개수:", len(df_dpo))

<제시문>
상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服) 절차에 대하여 논한 것이 신과는 큰 차이가 있었습니다 . 장자를 위하여 3년을 입는 까닭은 위로 ‘정체(正體)’가 되기 때문이고 또 전 중(傳重: 조상의 제사나 가문의 법통을 전함)하기 때문입니다 . …(중략) … 무엇보다 중요한 것은 할아버지와 아버지의 뒤를 이은 ‘정체’이지, 꼭 첫째이기 때문에 참 최 3년 복을 입는 것은 아닙니다 .”라고 하였다 .－현종실록 －ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 .ㄷ.정조 시기에 탕평 정치의 한 축을 이루었다 .ㄹ.이 이와 성혼의 문인을 중심으로 형성되었다.
<질문>
상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?

[0] Collected (A_Correct) | Total: 1
<제시문>
나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순조로워 우리나라 지맥의 뿌리가 되니 대업을 만대에 전할 땅이다. 왕은 춘하 추동네 계절의 중간달에 그곳에 가 100일 이상 머물러서 나라를 안녕케 하라. － 고려사－
<질문>
(가) 지역에 대한 설명으로 옳은 것은?

[1] Collected (A_Correct) | Total: 2
<제시문>
선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가)이/가 크게 노하여 한성부의 조례(皂隷)와 병졸로 하여 금 한 강 밖으로 몰아내게 하고 드디어 천여 곳의 서원을 철폐하고 그 토지를 몰수하여 관에 속하게 하였다.－대한계년사 －
<질문>
(가) 인물이 추진한 정책으로 옳지 않은 것은?

<제시문>
(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르기를 “나(대무 예)는 큰 나라를 맡아 여러 주변국을 다스렸으며, 고구려의 옛 땅을 회복하였고 부여의 풍속을 이었다.”라고 하였다.
<질문>
(가) 국가에 대한 설명으로 옳은 것은?

[3] Coll

KeyboardInterrupt: 

In [ ]:
dpo_results[0]

In [ ]:
df_dpo = pd.DataFrame(dpo_results)
df_dpo = df_dpo.loc[df_dpo['strategy'].isin(['Both_Correct_Judge_A','A_Correct'])]